In [1]:
import matplotlib
import pandas

In [2]:
#날씨 엑셀파일 열기
날씨 = pandas.read_excel('./data/날씨.xlsx')
날씨[:3]

,지역,날짜,일평균,기상
0,서울,2013-08-01,28.0,구름많음
1,서울,2013-08-02,27.4,흐림/안개/비
2,서울,2013-08-03,28.9,흐림/소나기


In [3]:
# 지역 열의 값이 서울인 항목만 데이터프레임에서 선택
서울선택 = 날씨["지역"] == "서울"
서울날씨 = 날씨[서울선택]

In [4]:
# 각 지역에서 평균 온도보다 높았던 날은 언제 일까요?
서울평균 = 서울날씨["일평균"].mean()
더운서울날 = 서울날씨["일평균"] > 서울평균
더운서울날.value_counts()

True     396
False    334
Name: 일평균, dtype: int64

In [6]:
서울날씨[더운서울날][:3]

,지역,날짜,일평균,기상
0,서울,2013-08-01,28.0,구름많음
1,서울,2013-08-02,27.4,흐림/안개/비
2,서울,2013-08-03,28.9,흐림/소나기


In [7]:
# 연습
# 수원날씨에 대해서도 1) 평균보다 더웠던 날들의 일 수 2) 그런 날들의 기록만 선택
수원선택 = 날씨["지역"] == "수원"
수원날씨 = 날씨[수원선택]
수원평균 = 수원날씨["일평균"].mean()
더운수원날 = 수원날씨["일평균"] > 수원평균
더운수원날씨 = 수원날씨[더운수원날]
더운수원날씨[:5]

,지역,날짜,일평균,기상
153,수원,2013-08-01,27.8,구름많음/안개
154,수원,2013-08-02,28.2,흐림/안개/비
155,수원,2013-08-03,28.6,구름많음/소나기
156,수원,2013-08-04,28.0,흐림/안개/비
157,수원,2013-08-05,27.9,구름많음/소나기/안개/천둥번개


In [8]:
## 여러 연도에 걸쳐서 기록이 수집된 것 같은데, 연도별 지역별로 집계를 보는 것도 필요하겠네
def 연도추출(날짜):
    return 날짜.year
연도 = 날씨['날짜'].map(lambda x : x.year)
연도별지역별 = 날씨.groupby([연도, "지역"])
# 연도별지역별["일평균"].mean()
연도별지역별["일평균"].mean().unstack()

지역,서울,수원
날짜,,
2013,14.241176,14.192810
2014,13.396712,13.145479
2015,12.680189,12.464151


In [9]:
# 두 개 이상의 기준으로 분류해서 표 모양으로 집계하는 것이 너무 흔한 기능이라 전용기능이 있음
날씨.pivot_table("일평균", index=연도, columns="지역")

지역,서울,수원
날짜,,
2013,14.241176,14.192810
2014,13.396712,13.145479
2015,12.680189,12.464151


In [12]:
# 월별 평균 기온을 구해보자
월정보 = 날씨['날짜'].map(lambda x : "{0}-{1:02d}".format(x.year, x.month))
월별날씨 = 날씨.pivot_table("일평균", index=월정보, columns="지역")

월별지역별 = 날씨.groupby([월정보, "지역"])
월별지역별["일평균"].mean().unstack()[:5]

지역,서울,수원
날짜,,
2013-08,27.696774,27.380645
2013-09,21.753333,21.720000
2013-10,15.774194,15.700000
2013-11,6.150000,6.153333
2013-12,-0.187097,-0.006452


In [13]:
## 요일별 지역별 일 평균 기존의 평균을 구해봅시다.
요일추출 = lambda x : ["월", "화", "수", "목","금","토","일"][x.weekday()]
요일별 = 날씨["날짜"].map(요일추출)
요일별지역별날씨 = 날씨.groupby([요일별, "지역"])
요일별지역별날씨["일평균"].mean().unstack()[:5]

지역,서울,수원
날짜,,
금,13.330476,13.135238
목,13.208571,13.053333
수,13.257692,13.094231
월,13.366346,13.239423
일,13.588462,13.374038


In [16]:
#요일 정렬
요일별지역별_평균 = 요일별지역별날씨["일평균"].mean()
원하는순서 = ["월", "화", "수", "목","금","토","일"]
요일별지역별_평균.ix[원하는순서].unstack()[:10]

지역,서울,수원
날짜,,
월,13.366346,13.239423
화,13.083654,12.900000
수,13.257692,13.094231
목,13.208571,13.053333
금,13.330476,13.135238
토,13.725962,13.375000
일,13.588462,13.374038


In [18]:
### 연습
# 식당xlsx 을 읽어 다음을 수행
# 1. 연도 별 코너 별 주문수량 평균 집계
# 2. 월별 코너별 주문 수량 평균 집계
# 3. 요일별 시간대별 주문수량 평균 집계
식당 = pandas.read_excel("./data/수정된식당.xlsx")
연도추출함수 = lambda x : x.year
월추출함수 = lambda x : "{0}-{1:02d}".format(x.year, x.month)
요일추출함수 = lambda x : ["월", "화", "수", "목","금","토","일"][x.weekday()]
연정보 = 식당["date"].map(연도추출함수)
월정보 = 식당["date"].map(월추출함수)
요일정보 = 식당["date"].map(요일추출함수)
연도별코너별주문수량평균 = 식당.pivot_table("주문수량", index=연정보, columns="코너")
월별코너별주문수량평균 = 식당.pivot_table("주문수량", index=월정보, columns="코너")
요일별코너별주문수량평균 = 식당.pivot_table("주문수량", index=요일정보, columns="코너")

In [19]:
# 출력
연도별코너별주문수량평균

코너,구이,면식,밥국1,밥국2,버거피자,야근,포장
date,,,,,,,
2013,261.616667,209.266667,323.147368,NaN,147.179191,160.355191,249.029536
2014,218.817610,175.964135,205.251034,240.285000,146.825902,143.287815,195.580027
2015,214.850746,164.936803,181.417476,244.340909,137.977695,123.577778,148.888620


In [20]:
월별코너별주문수량평균

코너,구이,면식,밥국1,밥국2,버거피자,야근,포장
date,,,,,,,
2013-08,299.565217,221.695652,330.976190,NaN,188.352941,154.333333,230.600000
2013-09,327.096774,224.419355,341.907407,NaN,156.700000,152.138889,244.528302
2013-10,266.571429,214.023810,329.539683,NaN,139.047619,157.190476,260.343284
2013-11,249.619048,209.309524,322.619048,NaN,146.452381,174.476190,267.692308
2013-12,199.547619,186.476190,295.984127,NaN,132.571429,159.452381,233.619048
2014-01,222.400000,199.625000,317.116667,NaN,142.282051,174.333333,249.383333
2014-02,220.282051,190.692308,279.155172,199.500000,140.948718,172.589744,254.368421
2014-03,221.809524,181.476190,203.682540,257.400000,153.500000,170.952381,252.161290
2014-04,224.829268,174.609756,190.919355,229.523810,146.309524,166.487805,243.714286


In [21]:
요일별코너별주문수량평균.ix[원하는순서]

코너,구이,면식,밥국1,밥국2,버거피자,야근,포장
date,,,,,,,
월,244.683673,187.189744,240.318792,261.014286,154.680628,139.819588,196.584775
화,244.911917,193.112821,228.658863,251.579710,148.000000,147.046154,197.315972
수,229.090909,186.150538,227.623239,238.085714,150.126374,143.739362,195.608696
목,222.727273,179.220339,219.099631,243.866667,147.468208,144.881356,190.030303
금,183.705202,146.694118,191.270370,212.412698,119.482759,128.285714,173.045455
토,NaN,NaN,NaN,NaN,NaN,NaN,NaN
일,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# 식당의 'date'열과 수원날씨의 '날짜'열을 기준으로 두 개의 데이터 프레임을 합치자
식당날씨 = pandas.merge(식당, 수원날씨, left_on='date', right_on='날짜')
# 합친 후 불필요한 컬럼 정리
식당날씨 = 식당날씨.drop(["지역", "date"], axis=1)
식당날씨[:5]

,시간대,코너,메뉴,is_sold_out,주문수량,pred_count,additional,good,ok,bad,diff_use_pred,year,month,day,wday,날짜,일평균,기상
0,저녁,구이,훈제연어벤또(200개),F,213,0,0,0,0,0,-213,2013,8,6,1,2013-08-06,25.9,구름많음/소나기/안개/천둥번개
1,저녁,밥국1,뚝배기 우거지 갈비탕,F,261,0,0,0,0,0,-261,2013,8,6,1,2013-08-06,25.9,구름많음/소나기/안개/천둥번개
2,점심,면식,판모밀정식,F,267,0,0,0,0,0,-267,2013,8,6,1,2013-08-06,25.9,구름많음/소나기/안개/천둥번개
3,아침,밥국1,설렁탕정식,F,1,0,0,0,0,0,-1,2013,8,6,1,2013-08-06,25.9,구름많음/소나기/안개/천둥번개
4,점심,밥국1,A. 부대찌개정식,F,376,0,0,0,0,0,-376,2013,8,6,1,2013-08-06,25.9,구름많음/소나기/안개/천둥번개


In [23]:
# 비온날, 안 온날 별 코너별 주문평균
비 = 식당날씨['기상'].str.contains('비')
코너별강수여부별_주문평균 = 식당날씨.pivot_table('주문수량', index='코너', columns=비)
코너별강수여부별_주문평균.rename(columns={True:'비', False : '안 옴'})

기상,안 옴,비
코너,,
구이,224.637410,230.104348
면식,177.933622,183.195652
밥국1,225.966355,209.840909
밥국2,238.848101,249.505263
버거피자,142.954745,148.285088
야근,142.327586,136.721030
포장,192.996139,184.162319
